In [4]:
#Imports

import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from consts import *

In [5]:
#GPU initialization for cnn learning

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU
Physical devices cannot be modified after being initialized


2022-06-06 13:36:50.856159: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 13:36:50.866349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 13:36:50.867191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 13:36:50.869703: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
#Images folders for train and testing
train_folder = '/home/historicalmanuscripts/Images_v2/train_otsu_3channels'
blind_folder = '/home/historicalmanuscripts/Images_v2/blind_test'

In [7]:
# Creating generators
# Train data set
train_file_names = np.array(os.listdir(train_folder))
class_labels = [const_classes[img.split("_")[0]] for img in train_file_names]
subclass_labels = [const_subclasses[img.split("_")[1]] for img in train_file_names]

# Split train to train and validation
C_train_data, C_valid_data, C_train_classes, C_valid_classes = train_test_split(train_file_names, class_labels, shuffle = True, stratify=class_labels)
SC_train_data, SC_valid_data, SC_train_classes, SC_valid_classes = train_test_split(train_file_names, subclass_labels, shuffle = True, stratify=subclass_labels)

# Creating generators instances
train_generator_C = Batch_Generator(train_folder, C_train_data, C_train_classes, batch_size)
train_generator_SC = Batch_Generator(train_folder, SC_train_data, SC_train_classes, batch_size)

validation_generator_C = Batch_Generator(train_folder, C_valid_data, C_valid_classes, batch_size)
validation_generator_SC = Batch_Generator(train_folder, SC_valid_data, SC_valid_classes, batch_size)

# Final generator instances w/o validation split
final_generator_C = Batch_Generator(train_folder, train_file_names, class_labels, batch_size)
final_generator_SC = Batch_Generator(train_folder, train_file_names, subclass_labels, batch_size)

In [8]:
#Generators for blind evalutation

blind_file_names = np.array(os.listdir(blind_folder))
blind_class_labels = np.array([const_classes[img.split("_")[0]] for img in blind_file_names if 'jpg' in img])
blind_subclass_labels = np.array([const_subclasses[img.split("_")[1]] for img in blind_file_names if 'jpg' in img])

blind_generator_C = Batch_Generator(blind_folder, blind_file_names, blind_class_labels, batch_size)
blind_generator_SC = Batch_Generator(blind_folder, blind_file_names, blind_subclass_labels, batch_size)

In [10]:
#Class weight distribution for weighted learing

total = len(train_file_names)
class_weights = {c:(1 / class_labels.count(c)) * (total / 2.0) for c in const_classes.values()}
subclass_weights = {c:(1 / subclass_labels.count(c)) * (total / 2.0) for c in const_subclasses.values()}

print(class_weights)
print(subclass_weights)

{0: 2.395328338475099, 1: 14.97245179063361, 2: 2.6668302257114815, 3: 4.334130781499202, 4: 5.112888052681091, 5: 1.3996909605974763}
{0: 4.465899753492193, 1: 1.1264248704663213, 2: 1.125724937862469}


In [ ]:
#History dict and results for plotting
final_history_dict = {}
results = {'class':{}, 'subclass':{}}
data = preprocess_data(blind_folder,'inv_otsu')

In [11]:
#EfficientNetV2B2
#20 ephocs
#non weighted classes
#dropout 0.2
model = 'EfficientNetV2B2'
print(model)
m = create_model(model,input_shape,originNum)
m.summary()
final_history_dict[model] = m.fit(x=final_generator_C,
                          epochs=20)
m.save(model+"_final_20_dropout02")

EfficientNetV2B2
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 efficientnetv2-b2 (Function  (None, 13, 13, 1408)     8769374   
 al)                                                             
                                                                 
 global_average_pooling2d_1   (None, 1408)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 6)                 8454      
                                                                 
Total params: 8,777,828
Trainable params: 8,454
Non-trainable params: 8,769,374
_________________________________________________________________
Epoch 1/20
340/340 [========

2022-06-06 14:27:47.987004: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: EfficientNetV2B2_final_20_dropout02/assets


In [12]:
#EfficientNetV2B2
#20 ephocs
#weighted classes
#dropout 0.2
model = 'EfficientNetV2B2'
print(model)
m = create_model(model,input_shape,originNum)
m.summary()
final_history_dict[model] = m.fit(x=final_generator_C,
                          epochs=20,
                          class_weight = class_weights)
m.save(model+"_final_20_dropout02_weighted")

EfficientNetV2B2
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 efficientnetv2-b2 (Function  (None, 13, 13, 1408)     8769374   
 al)                                                             
                                                                 
 global_average_pooling2d_2   (None, 1408)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 6)                 8454      
                                                                 
Total params: 8,777,828
Trainable params: 8,454
Non-trainable params: 8,769,374
_________________________________________________________________
Epoch 1/20
340/340 [========

In [13]:
#Xception
#20 ephocs
#non weighted classes
#dropout 0.2
model = 'Xception'
print(model)
m = create_model(model,input_shape,styleNum)
m.summary()
final_history_dict[model+'SC'] = m.fit(x=final_generator_SC,
                          epochs=20)
m.save(model+"_SC_final_20_dropout02")

Xception
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 400, 400, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 400, 400, 3)      0         
 a)                                                              
                                                                 
 xception (Functional)       (None, 13, 13, 2048)      20861480  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                  

In [14]:
#Xception
#20 ephocs
#weighted classes
#dropout 0.2
model = 'Xception'
print(model)
m = create_model(model,input_shape,styleNum)
m.summary()
final_history_dict[model+'SC'] = m.fit(x=final_generator_SC,
                          epochs=20,
                          class_weight = subclass_weights)
m.save(model+"_SC_final_20_dropout02_weighted")

Xception
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 400, 400, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 400, 400, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 400, 400, 3)      0         
 bda)                                                            
                                                                 
 xception (Functional)       (None, 13, 13, 2048)      20861480  
                                                                 
 global_average_pooling2d_4   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                  

In [16]:
model = 'EfficientNetV2B2_final_20_dropout02'
print(model,' class:\n')
results['class'][model] = evaluate_model(data, tf.keras.models.load_model(model),'class', True)
model = 'EfficientNetV2B2_final_20_dropout02_weighted'
print(model,' class:\n')
results['class'][model] = evaluate_model(data, tf.keras.models.load_model(model),'class', True)


model = 'Xception_SC_final_20_dropout02'
print(model,' class:\n')
results['subclass'][model] = evaluate_model(data, tf.keras.models.load_model(model),'subclass', True)
model = 'Xception_SC_final_20_dropout02_weighted'
print(model,' class:\n')
results['subclass'][model] = evaluate_model(data, tf.keras.models.load_model(model),'subclass', True)

EfficientNetV2B2_final_20_dropout02  class:

Image 171, Stats: 80/172

EfficientNetV2B2_final_20_dropout02_weighted  class:

Image 171, Stats: 83/172

Xception_SC_final_20_dropout02  class:

Image 171, Stats: 107/172

Xception_SC_final_20_dropout02_weighted  class:

Image 171, Stats: 99/172



In [18]:
model = 'EfficientNetV2B2_final'
print(model,' class:\n')
results['class'][model] = evaluate_model(data, tf.keras.models.load_model(model),'class', True)
model = 'Xception_SC_final'
print(model,' class:\n')
results['subclass'][model] = evaluate_model(data, tf.keras.models.load_model(model),'subclass', True)
model = 'EfficientNetV2B2_final_50'
print(model,' class:\n')
results['class'][model] = evaluate_model(data, tf.keras.models.load_model(model),'class', True)
model = 'Xception_SC_final_50'
print(model,' class:\n')
results['subclass'][model] = evaluate_model(data, tf.keras.models.load_model(model),'subclass', True)
model = 'EfficientNetV2B2_final_100'
print(model,' class:\n')
results['class'][model] = evaluate_model(data, tf.keras.models.load_model(model),'class', True)
model = 'Xception_SC_final_100'
print(model,' class:\n')
results['subclass'][model] = evaluate_model(data, tf.keras.models.load_model(model),'subclass', True)

EfficientNetV2B2_final  class:

Image 171, Stats: 94/172

Xception_SC_final  class:

Image 171, Stats: 105/172

EfficientNetV2B2_final_50  class:

Image 171, Stats: 90/172

Xception_SC_final_50  class:

Image 171, Stats: 110/172

EfficientNetV2B2_final_100  class:

Image 171, Stats: 84/172

Xception_SC_final_100  class:

Image 171, Stats: 101/172



In [19]:
model = 'EfficientNetV2B2_final_100_weighted'
print(model,' class:\n')
results['subclass'][model] = evaluate_model(data, tf.keras.models.load_model(model),'subclass', True)
model = 'Xception_SC_final_100_weighted'
print(model,' class:\n')
results['subclass'][model] = evaluate_model(data, tf.keras.models.load_model(model),'subclass', True)

EfficientNetV2B2_final_100_weighted  class:

Image 171, Stats: 23/172

Xception_SC_final_100_weighted  class:

Image 171, Stats: 102/172

